# 一、数据导入

In [1]:
import pandas as pd
df=pd.read_excel('分类数据集.xlsx')
X,y=df.iloc[:,:-1],df.iloc[:,-1]# X有12个特征，y有4列
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2023,shuffle=True)

In [2]:
X_train.describe()# 可见特征间的量级差异较大，需要预处理。

,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,特征10,特征11,特征12
count,285.000000,285.000000,285.000000,285.000000,285.000000,2.850000e+02,285.000000,285.000000,285.000000,285.000000,285.000000,285.000000
mean,4.193885,2.372686,39.657695,24.140837,40277.921077,3.446019e+06,5.367561,71.440205,0.000027,0.000090,-833.080818,43.641683
std,0.009229,0.251619,1.177859,0.461651,3225.803189,1.734785e+05,1.566913,14.205196,0.000013,0.000088,862.406028,87.139548
min,4.172034,1.737030,37.080390,22.969923,34768.857013,3.162616e+06,3.172332,45.616416,0.000004,0.000007,-2503.187682,-44.110979
25%,4.186483,2.140416,38.751283,23.878754,37394.823886,3.340732e+06,4.321113,63.487632,0.000016,0.000029,-1454.557874,-6.631219
50%,4.197855,2.417717,39.586755,24.107818,40151.703391,3.397332e+06,5.052645,74.870794,0.000027,0.000058,-447.041714,4.414913
75%,4.200966,2.631344,40.584997,24.406482,43178.125224,3.492656e+06,6.091445,80.659888,0.000035,0.000136,-156.562330,37.310688
max,4.233325,2.699288,42.239987,26.647374,46723.171766,4.017925e+06,8.305601,95.336050,0.000056,0.000453,-70.780981,281.757710


# 二、数据预处理

In [3]:
from sklearn.preprocessing import Binarizer,MaxAbsScaler,MinMaxScaler,Normalizer,StandardScaler,RobustScaler
# transformer = Binarizer(threshold=0.0,copy=True)# 二值化
# transformer = MaxAbsScaler(copy=True)#除以每一列的最大值
# transformer=MinMaxScaler(feature_range=(0,1),clip=False,copy=True)#把值定到【0，1】闭区间，
# transformer=Normalizer(norm='l2',copy=True)
t=StandardScaler(with_mean=True,with_std=True)
# transformer=RobustScaler

In [4]:
X_train_t=t.fit_transform(X_train)
X_test_t=t.transform(X_test)

# 三、降维

In [5]:
## 方式一
from sklearn.decomposition import PCA
dd=PCA(n_components=5)
## 方式二
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
dd=LinearDiscriminantAnalysis(n_components=5)
## 方式组三
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,mutual_info_classif,chi2#分类
from sklearn.feature_selection import f_regression,mutual_info_regression#回归
dd=SelectKBest(score_func=f_classif,k=5)
## 方式组四   按比例
from sklearn.feature_selection import SelectPercentile
dd=SelectPercentile(score_func=f_classif,percentile=50)

In [6]:
X_train_t_dd=dd.fit_transform(X_train_t,y_train)
# X_train_t_dd=dd.fit_transform(X_train_t)#方式一，不需要类别信息
X_test_t_dd=dd.transform(X_test_t)

# 四、聚类

In [7]:
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.cluster import DBSCAN,AgglomerativeClustering,KMeans
cls=DBSCAN(eps=1.1, min_samples=2)
cls=AgglomerativeClustering(n_clusters=3, linkage='single')
cls=KMeans(n_clusters=3,
           init='k-means++',#np.array([[0, 0], [0, 1], [3, 1]])可以自定义
           n_init=10,#重复10次取平均
          )

In [8]:
cls.fit(X_train_t_dd)
#计算轮廓系数
# s_score=silhouette_score(X_train_t_dd,y_train)
# s_sample=silhouette_score(X_train_t_dd,y_train)

KMeans(n_clusters=3)

# 五、分类

In [9]:
# 以KNN为例
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train_t_dd,y_train)

KNeighborsClassifier()

In [10]:
y_pred=clf.predict(X_test_t_dd)

# 六、评价

In [11]:
from sklearn.metrics import accuracy_score,recall_score, precision_score, f1_score
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      1.00      0.95        18
           2       0.94      0.89      0.91        18
           3       1.00      0.85      0.92        20
           4       0.89      1.00      0.94        16

    accuracy                           0.93        72
   macro avg       0.93      0.93      0.93        72
weighted avg       0.94      0.93      0.93        72



In [12]:
print(classification_report(y_test,y_pred,output_dict=True)['1'])

{'precision': 0.9, 'recall': 1.0, 'f1-score': 0.9473684210526316, 'support': 18}


# 七、模型发布

In [15]:
t=StandardScaler(with_mean=True,with_std=True)
dd=SelectPercentile(score_func=f_classif,percentile=50)
clf=KNeighborsClassifier(n_neighbors=5)
X_=t.fit_transform(X)
X__=dd.fit_transform(X_,y)
clf.fit(X__,y)

KNeighborsClassifier()